In [0]:
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

def collect_quality_metrics(schema_name="silver"):
    tables = spark.sql(f"SHOW TABLES IN first_phase.{schema_name}").collect()
    metrics = []

    for table_row in tables:
        table_name = table_row.tableName
        full_table = f"first_phase.{schema_name}.{table_name}"
        df = spark.table(full_table)

        # Загальна кількість рядків
        total_count = df.count()

        # Отримуємо попереднє значення row_count, якщо існує
        prev = spark.sql(f"""
            SELECT metric_value 
            FROM first_phase.monitoring.data_quality_metrics
            WHERE table_name = '{table_name}' 
              AND metric_name = 'total_rows'
            ORDER BY check_timestamp DESC
            LIMIT 1
        """).collect()

        prev_count = prev[0]["metric_value"] if prev else None
        diff_pct = abs(total_count - prev_count) / prev_count * 100 if prev_count and prev_count > 0 else 0
        status = "WARNING" if diff_pct > 20 else "OK"

        metrics.append({
            'table_name': table_name,
            'metric_name': 'total_rows',
            'metric_value': float(total_count),
            'check_timestamp': datetime.now(),
            'status': status,
            'details': f'Row count changed by {diff_pct:.2f}% from last check' if prev_count else 'Initial check'
        })

        # NULL у ключових полях
        for column in df.columns:
            if 'key' in column.lower() or column.endswith('_id'):
                null_count = df.filter(F.col(column).isNull()).count()
                null_pct = (null_count / total_count * 100) if total_count > 0 else 0

                metrics.append({
                    'table_name': table_name,
                    'metric_name': f'null_pct_{column}',
                    'metric_value': float(null_pct),
                    'check_timestamp': datetime.now(),
                    'status': 'ERROR' if null_count > 0 else 'OK',
                    'details': f'{null_count} null values found in {column}'
                })

        #  Перевірка дублікованих PK

        pk_cols = [c for c in df.columns if c.lower().endswith('key') or c.endswith('_id')]
        if len(pk_cols) == 1:  # якщо в таблиці один PK
            pk_col = pk_cols[0]
            dup_count = df.groupBy(pk_col).count().filter("count > 1").count()

            metrics.append({
                'table_name': table_name,
                'metric_name': f'duplicated_{pk_col}',
                'metric_value': float(dup_count),
                'check_timestamp': datetime.now(),
                'status': 'ERROR' if dup_count > 0 else 'OK',
                'details': f'{dup_count} duplicated primary keys in {pk_col}'
            })

    # Запис метрик у таблицю
    metrics_df = spark.createDataFrame(metrics)
    metrics_df.write.mode("append").saveAsTable("first_phase.monitoring.data_quality_metrics")


In [0]:

def capture_schema_snapshot(schema_name="silver"):
    """
    Зберігаємо snapshot схеми для виявлення змін
    """
    print(f"\n📸 Capturing schema snapshot for {schema_name}...")
    
    tables = spark.sql(f"SHOW TABLES IN {CATALOG}.{schema_name}").collect()
    snapshot_count = 0

    for table_row in tables:
        table_name = table_row.tableName
        columns = spark.table(f"{CATALOG}.{schema_name}.{table_name}").dtypes

        for col_name, col_type in columns:
            spark.sql(f"""
                INSERT INTO {CATALOG}.monitoring.schema_snapshots 
                VALUES ('{table_name}', '{col_name}', '{col_type}', current_timestamp())
            """)
            snapshot_count += 1
    
    print(f"   ✅ Captured {snapshot_count} column definitions")

In [0]:
def check_referential_integrity():
    checks = [
        ("orders", "o_custkey", "customer", "c_custkey"),
        ("lineitem", "l_orderkey", "orders", "o_orderkey"),
        ("partsupp", "ps_partkey", "part", "p_partkey"),
        ("supplier", "s_nationkey", "nation", "n_nationkey"),
    ]

    for child_tbl, fk, parent_tbl, pk in checks:
        orphaned = spark.sql(f"""
            SELECT COUNT(*) AS cnt
            FROM first_phase.silver.{child_tbl} c
            LEFT JOIN first_phase.silver.{parent_tbl} p
              ON c.{fk} = p.{pk}
            WHERE p.{pk} IS NULL
        """).collect()[0]['cnt']

        spark.sql(f"""
            INSERT INTO first_phase.monitoring.data_quality_metrics
            VALUES (
                '{child_tbl}',
                'orphaned_fk_{fk}',
                {float(orphaned)},
                current_timestamp(),
                '{'ERROR' if orphaned > 0 else 'OK'}',
                'Found {orphaned} orphaned foreign keys from {child_tbl}.{fk} → {parent_tbl}.{pk}'
            )
        """)


In [0]:
def log_load_info(schema, table, rows, duration, status):
    spark.sql(f"""
        INSERT INTO {CATALOG}.monitoring.load_history 
        VALUES ('{schema}', '{table}', current_timestamp(), {rows}, {duration}, '{status}')
    """)

In [0]:
def run_monitoring():
    print("Starting data quality monitoring...")
    collect_quality_metrics("bronze")
    collect_quality_metrics("silver")
    check_referential_integrity()
    capture_schema_snapshot("silver")
    print("Monitoring completed!")

In [0]:
run_monitoring()

In [0]:
display(spark.sql("SELECT * FROM first_phase.monitoring.data_quality_metrics ORDER BY check_timestamp DESC"))
